In [1]:
import numpy as np
from scipy.optimize import minimize

In [2]:
# Исходные данные
x0 = np.array([2.0, 4.0])
epsilon = 0.05
alpha = 0.01
k = 1.0
m = 2.0  # коэффициент увеличения штрафа
max_iter = 3  # количество итераций

# Целевая функция
def f(x):
    return 4 * x[0] - x[1]**2 - 12

# Ограничения
def g1(x):
    return 10 * x[0] - x[0]**2 + 10 * x[1] - x[1]**2 - 34

def g2(x):
    return x[0]

def g3(x):
    return x[1]

# Градиент целевой функции
def grad_f(x):
    return np.array([4, -2 * x[1]])

# Градиенты ограничений
def grad_g1(x):
    return np.array([10 - 2 * x[0], 10 - 2 * x[1]])

def grad_g2(x):
    return np.array([1, 0])

def grad_g3(x):
    return np.array([0, 1])

# Функция штрафа (логарифмическая барьерная функция)
def penalty_function(x, k):
    phi = 0.0
    if g1(x) <= 0:
        phi += -np.log(g1(x))
    if g2(x) <= 0:
        phi += -np.log(g2(x))
    if g3(x) <= 0:
        phi += -np.log(g3(x))
    return f(x) + k * phi

# Градиент функции штрафа
def grad_penalty(x, k):
    grad_phi = np.zeros(2)
    if g1(x) <= 0:
        grad_phi += -grad_g1(x) / g1(x)
    if g2(x) <= 0:
        grad_phi += -grad_g2(x) / g2(x)
    if g3(x) <= 0:
        grad_phi += -grad_g3(x) / g3(x)
    return grad_f(x) + k * grad_phi

# Метод штрафных функций
def penalty_method(x0, k, alpha, epsilon, m, max_iter):
    x = x0.copy()
    iter_count = 0
    
    for _ in range(max_iter):
        grad_P = grad_penalty(x, k)
        grad_norm = np.linalg.norm(grad_P)
        
        print(f"Итерация {iter_count + 1}:")
        print(f"x = {x}, P(x, k) = {penalty_function(x, k)}, ||∇P|| = {grad_norm}")
        
        if grad_norm <= epsilon:
            break
        
        x_new = x - alpha * grad_P
        k = m * k
        iter_count += 1
        x = x_new
    
    return x

# Запуск метода
result = penalty_method(x0, k, alpha, epsilon, m, max_iter)
print("\nРезульат:")
print(f"x = {result}, f(x) = {f(result)}")

Итерация 1:
x = [2. 4.], P(x, k) = -20.0, ||∇P|| = 8.94427190999916
Итерация 2:
x = [1.96 4.08], P(x, k) = -20.8064, ||∇P|| = 9.087661965544273
Итерация 3:
x = [1.92   4.1616], P(x, k) = -21.63891456, ||∇P|| = 9.23448202337305

Резульат:
x = [1.88     4.244832], f(x) = -22.498598708223998


In [3]:
# Параметры
x = np.array([2.0, 4.0])  # начальное приближение
epsilon = 1e-4  # точность по f(x)
alpha = 0.05  # начальный шаг
max_inner_iter = 100
t = 1.0  # начальный параметр барьера
mu = 10  # множитель для роста t

# Целевая функция и её градиент
def f(x):
    return 4 * x[0] - x[1]**2 - 12

def grad_f(x):
    return np.array([4, -2 * x[1]])

# Ограничения и градиенты
def g1(x): return 10 * x[0] - x[0]**2 + 10 * x[1] - x[1]**2 - 34
def g2(x): return x[0]
def g3(x): return x[1]

def grad_g1(x): return np.array([10 - 2 * x[0], 10 - 2 * x[1]])
def grad_g2(x): return np.array([1, 0])
def grad_g3(x): return np.array([0, 1])

# Барьерная функция и её градиент
def barrier(x):
    if g1(x) <= 0 or g2(x) <= 0 or g3(x) <= 0:
        return np.inf
    return - (np.log(g1(x)) + np.log(g2(x)) + np.log(g3(x)))

def grad_barrier(x):
    if g1(x) <= 0 or g2(x) <= 0 or g3(x) <= 0:
        return np.array([np.inf, np.inf])
    return - (
        grad_g1(x) / g1(x) +
        grad_g2(x) / g2(x) +
        grad_g3(x) / g3(x)
    )

# Комбинированная функция и градиент
def phi(x, t):
    return t * f(x) + barrier(x)

def grad_phi(x, t):
    return t * grad_f(x) + grad_barrier(x)

# Метод барьерных функций
for outer_iter in range(3):  # 3 "внешних" шага
    print(f"\n=== Итерация {outer_iter + 1} ===")
    for inner_iter in range(max_inner_iter):
        grad = grad_phi(x, t)
        if np.any(np.isinf(grad)):
            print("x вне допустимой области, остановка.")
            break

        # Предлагаем шаг
        step = alpha * grad
        x_candidate = x - step

        # Backtracking для корректного x
        backtrack = 0
        while (g1(x_candidate) <= 0 or g2(x_candidate) <= 0 or g3(x_candidate) <= 0) and backtrack < 10:
            alpha *= 0.5
            step = alpha * grad
            x_candidate = x - step
            backtrack += 1

        if backtrack == 10:
            print("Не удалось найти допустимый шаг.")
            break

        # Условие остановки по цели
        if abs(f(x_candidate) - f(x)) < epsilon:
            x = x_candidate
            print("Условие остановки выполнено.")
            break

        x = x_candidate

    print(f"x = {x/2}, f(x) = {f(x/2)}")
    t *= mu  # усиливаем фокус на f(x), уменьшая влияние барьера

print("\n=== Результат после 3 итераций ===")
print(f"x = {x/2}, f(x) = {f(x/2)}")



=== Итерация 1 ===
x = [2.11044976 4.42641267], f(x) = -23.15133011581439

=== Итерация 2 ===
x = [2.09053037 4.44781247], f(x) = -23.420914311894165

=== Итерация 3 ===
x = [2.08759243 4.45683762], f(x) = -23.513031841981174

=== Результат после 3 итераций ===
x = [2.08759243 4.45683762], f(x) = -23.513031841981174
